<a href="https://colab.research.google.com/github/MayerT1/ACI_Collaboration/blob/main/ACI_cerulean_warbler_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pygbif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 4.8 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import pandas as pd
from pygbif import occurrences
import geemap.chart as chart

In [3]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='servir-sco-assets')
Map = geemap.Map()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


https://www.gbif.org/species/6093350

In [4]:
# coords = [
#     [-85.85307177259287, 35.06788356661989],
#     [-85.41087206556162, 35.06788356661989],
#     [-85.41087206556162, 35.34391870157585],
#     [-85.85307177259287, 35.34391870157585],
#     [-85.85307177259287, 35.06788356661989]  # Close the polygon
# ]

# roi = ee.Geometry.Polygon(coords)

# # Create a map centered on the ROI
# Map = geemap.Map(center=[35.2, -85.63], zoom=11)
# Map.addLayer(roi, {"color": "red"}, "Modeling ROI")
# # Map

In [5]:
# # Define AOI: Tennessee
# states = ee.FeatureCollection("TIGER/2018/States")
# tennessee = states.filter(ee.Filter.eq('NAME', 'Tennessee'))
# roi = tennessee.geometry()
# Map = geemap.Map(center=[35, -86], zoom=6)
# Map.addLayer(tennessee, {}, "Tennessee AOI")

# # --------------------------------------------------
# # Fetch Cerulean Warbler occurrences from GBIF
# # --------------------------------------------------
# species_name = "Setophaga cerulea"
# gbif_data = occurrences.search(scientificName=species_name, country="US", stateProvince="Tennessee", hasCoordinate=True, limit=3000)

# # Convert GBIF occurrences to a DataFrame
# records = gbif_data['results']
# df = pd.DataFrame(records)
# df = df.dropna(subset=['decimalLatitude', 'decimalLongitude'])

# # Convert to ee.FeatureCollection
# def df_to_fc(row):
#     return ee.Feature(ee.Geometry.Point([row['decimalLongitude'], row['decimalLatitude']]), {'occurrence': 1})

# presence_fc = ee.FeatureCollection(df.apply(df_to_fc, axis=1).tolist())
# Map.addLayer(presence_fc, {'color': 'blue'}, "Presence Points")

# # --------------------------------------------------
# # Generate background (pseudo-absence) points
# # --------------------------------------------------
# background_fc = ee.FeatureCollection.randomPoints(roi, 10000, seed=42)
# background_fc = background_fc.map(lambda f: f.set('occurrence', 0))
# Map.addLayer(background_fc, {'color': 'red'}, "Background Points")

# # Merge presence and background points
# training_data = presence_fc.merge(background_fc)
# Map.addLayer(training_data, {}, "Training Data")

# # --------------------------------------------------
# # Landsat 8 imagery preprocessing
# # --------------------------------------------------
# def maskL8sr(image):
#     qa = image.select('QA_PIXEL')
#     cloudShadowBitMask = 1 << 3
#     cloudsBitMask = 1 << 5
#     mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
#         qa.bitwiseAnd(cloudsBitMask).eq(0)
#     )
#     return image.updateMask(mask)

# LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
#         .filterDate('2020-01-01', '2020-12-31') \
#         .filterBounds(roi) \
#         .map(maskL8sr)

# # Compute NDVI median image
# def add_ndvi(image):
#     ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
#     return ndvi.copyProperties(image).set('system:time_start', image.get('system:time_start'))

# image = LS8.map(add_ndvi).median().clip(roi)
# Map.addLayer(image, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, "NDVI")

# # --------------------------------------------------
# # Sample points and train Maxent classifier
# # --------------------------------------------------
# training = image.sampleRegions(
#     collection=training_data,
#     scale=30
# )

# classifier = ee.Classifier.amnhMaxent().train(
#     features=training,
#     classProperty='occurrence',
#     inputProperties=image.bandNames()
# )

# # Classify image
# image_classified = image.classify(classifier)
# # Map.addLayer(image_classified.select('probability'), {'min':0, 'max':1, 'palette': ['white','light green','green']}, 'Predicted Probability')

# # Define a gradient from transparent to dark green
# palette = ['#00ffffff', '#006400ff']  # transparent -> dark green

# # Add layer
# Map.addLayer(image_classified.select('probability'),
#              {'min':0, 'max':1, 'palette': palette},
#              'Predicted Probability')



# # Display map
# # Map


In [6]:
# # import ee
# # import geemap
# # import pandas as pd
# # from pygbif import occurrences

# # ee.Initialize()

# # -----------------------------
# # Define AOI: Tennessee
# # -----------------------------
# states = ee.FeatureCollection("TIGER/2018/States")
# tennessee = states.filter(ee.Filter.eq('NAME', 'Tennessee'))
# roi = tennessee.geometry()
# Map = geemap.Map(center=[35, -86], zoom=6)
# Map.addLayer(tennessee, {}, "Tennessee AOI")

# # # Define your custom ROI
# # coords = [
# #     [-85.85307177259287, 35.06788356661989],
# #     [-85.41087206556162, 35.06788356661989],
# #     [-85.41087206556162, 35.34391870157585],
# #     [-85.85307177259287, 35.34391870157585],
# #     [-85.85307177259287, 35.06788356661989]  # Close the polygon
# # ]

# # # Create EE Geometry and buffer by 10 km (10000 meters)
# # roi = ee.Geometry.Polygon(coords).buffer(100000)


# # -----------------------------
# # GBIF occurrences for Cerulean Warbler
# # -----------------------------
# species_name = "Setophaga cerulea"
# gbif_data = occurrences.search(scientificName=species_name, country="US", stateProvince="Tennessee", hasCoordinate=True, limit=3000)
# records = gbif_data['results']
# df = pd.DataFrame(records).dropna(subset=['decimalLatitude', 'decimalLongitude'])

# def df_to_fc(row):
#     return ee.Feature(ee.Geometry.Point([row['decimalLongitude'], row['decimalLatitude']]), {'occurrence': 1})

# presence_fc = ee.FeatureCollection(df.apply(df_to_fc, axis=1).tolist())
# Map.addLayer(presence_fc, {'color': 'blue'}, "Presence Points")

# # -----------------------------
# # Background points
# # -----------------------------
# background_fc = ee.FeatureCollection.randomPoints(roi, 10000, seed=42).map(lambda f: f.set('occurrence', 0))
# training_data = presence_fc.merge(background_fc)
# Map.addLayer(background_fc, {'color': 'red'}, "Background Points")

# # -----------------------------
# # Landsat 8 NDVI
# # -----------------------------
# def maskL8sr(image):
#     qa = image.select('QA_PIXEL')
#     cloudShadowBitMask = 1 << 3
#     cloudsBitMask = 1 << 5
#     mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
#         qa.bitwiseAnd(cloudsBitMask).eq(0)
#     )
#     return image.updateMask(mask)

# LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
#         .filterDate('2020-01-01', '2020-12-31') \
#         .filterBounds(roi) \
#         .map(maskL8sr)

# def add_ndvi(image):
#     return image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# ndvi_image = LS8.map(add_ndvi).median().clip(roi)
# Map.addLayer(ndvi_image, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, "NDVI")

# # -----------------------------
# # DEM
# # -----------------------------
# dem = ee.Image("USGS/SRTMGL1_003").clip(roi).rename("DEM")
# Map.addLayer(dem, {'min': 0, 'max': 2000, 'palette': ['white','brown']}, "DEM")

# # -----------------------------
# # Climate: WorldClim v2 bioclimatic variables (1km resolution)
# # -----------------------------
# bio1 = ee.Image("WORLDCLIM/V1/BIO").select('bio01').clip(roi).rename("bio01")  # Mean annual temp
# bio12 = ee.Image("WORLDCLIM/V1/BIO").select('bio12').clip(roi).rename("bio12")  # Annual precipitation

# # -----------------------------
# # Forest canopy ]
# # -----------------------------
# # # TCC and TCH from Global Forest Canopy Height and Cover
# # tcc = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").clip(roi).rename("TCC")  # approximate
# # # GEDI products
# tcc = ee.ImageCollection("projects/sat-io/open-datasets/facebook/meta-canopy-height").first().clip(roi).rename("TCC");

# # -----------------------------
# # Forest canopy / GEDI L2A metrics
# # -----------------------------

# gedi_l2a_fc = ee.FeatureCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY") \
#                 .filterBounds(roi) \
#                 .filter(ee.Filter.date('2019-01-01', '2020-12-31'))


# gedi_rh98_img = gedi_l2a_fc.reduceToImage(
#     properties=['rh98'],
#     reducer=ee.Reducer.mean()
# ).clip(roi).rename('Rh98')  # no select()


# # gedi_rh98_img = gedi_rh98_img.focal_mean(radius=30, units='meters')


# # -----------------------------
# # Forest canopy / GEDI L2B metrics
# # -----------------------------

# gedi_l2b_fc = ee.FeatureCollection("LARSE/GEDI/GEDI02_B_002_MONTHLY") \
#                 .filterBounds(roi) \
#                 .filter(ee.Filter.date('2019-01-01', '2020-12-31'))


# gedi_pai_img = gedi_l2b_fc.reduceToImage(
#     properties=['pai'],
#     reducer=ee.Reducer.mean()
# ).clip(roi).rename('PAI')  # no select()


# # -----------------------------
# # Define square kernel for smoothing
# # -----------------------------
# kernel = ee.Kernel.square(radius=100)  # 100 m like your JS example

# # Apply focal mean directly to single-band images
# gedi_pai_img_smooth = gedi_pai_img.focal_mean(kernel=kernel, iterations=1)
# gedi_rh98_img_smooth = gedi_rh98_img.focal_mean(kernel=kernel, iterations=1)


# # gedi_pai_img = gedi_pai_img.focal_mean(radius=30, units='meters')


# # # List of PAVDZ bands
# # # pai_bands = [f'pai_z{i}' for i in range(15)]
# # pavdz_bands = [f'pavdz_z{i}' for i in range(15)]


# # def rasterize_bands(fc, band_list):
# #     """Convert a list of point-based bands to a raster image stack."""
# #     raster_list = []
# #     for band in band_list:
# #         img = fc.reduceToImage(properties=[band], reducer=ee.Reducer.mean()) \
# #                 .select(['mean']).clip(roi) \
# #                 .rename(band) \
# #                 .focal_mean(radius=30, units='meters')  # smooth gaps
# #         raster_list.append(img)
# #     return ee.Image(raster_list)


# # # Rasterize PAI and PAVDZ
# # # gedi_pai_img = rasterize_bands(gedi_l2b_fc, pai_bands)
# # gedi_pavdz_img = rasterize_bands(gedi_l2b_fc, pavdz_bands)

# # -----------------------------
# # Rasterize GEDI bands with square kernel smoothing
# # -----------------------------

# def rasterize_bands_smooth(fc, band_list, kernel_radius=100):
#     """Convert point-based bands to raster and apply focal mean using a square kernel."""
#     raster_list = []

#     # Define square kernel (like JS example)
#     kernel = ee.Kernel.square(radius=kernel_radius)

#     for band in band_list:
#         img = fc.reduceToImage(properties=[band], reducer=ee.Reducer.mean()) \
#                 .select(['mean']).clip(roi) \
#                 .rename(band)
#         # Apply focal mean using the kernel
#         img_smooth = img.focal_mean(kernel=kernel, iterations=1)
#         raster_list.append(img_smooth)

#     # Combine all smoothed rasters into a single image
#     return ee.Image(raster_list)


# # List of PAVDZ bands
# pavdz_bands = [f'pavdz_z{i}' for i in range(10)]

# # Rasterize and smooth
# gedi_pavdz_img_out = rasterize_bands_smooth(gedi_l2b_fc, pavdz_bands, kernel_radius=100)

# # -----------------------------
# # Combine all predictors into a single image
# # -----------------------------

# predictors = ndvi_image.addBands([dem, bio1, bio12, tcc, gedi_pavdz_img_out])#, gedi_pai_img_smooth, gedi_rh98_img_smooth])
# Map.addLayer(predictors, {}, "All Predictors")

# # -----------------------------
# # Sample points and train Maxent classifier
# # -----------------------------
# training = predictors.sampleRegions(
#     collection=training_data,
#     scale=30
# )

# classifier = ee.Classifier.amnhMaxent().train(
#     features=training,
#     classProperty='occurrence',
#     inputProperties=predictors.bandNames()
# )

# # Classify image
# image_classified = predictors.classify(classifier)
# Map.addLayer(image_classified.select('probability'), {'min':0, 'max':1, 'palette': ['white','light green','green']}, 'Predicted Probability')

# Map


wall to wall dev

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
!mkdir -p "/content/drive/MyDrive/ACI_Geo_Data"

In [45]:
%cd "/content/drive/MyDrive/ACI_Geo_Data"

/content/drive/MyDrive/ACI_Geo_Data


In [46]:
!pwd

/content/drive/MyDrive/ACI_Geo_Data


In [11]:
# -----------------------------
# Define AOI: Tennessee
# -----------------------------
# states = ee.FeatureCollection("TIGER/2018/States")
# tennessee = states.filter(ee.Filter.eq('NAME', 'Tennessee'))
# roi = tennessee.geometry()
# Map = geemap.Map(center=[35, -86], zoom=6)
# Map.addLayer(tennessee, {}, "Tennessee AOI")


# -----------------------------
# Smaller ROI around custom point
# -----------------------------
point = ee.Geometry.Point([-85.65568438327509, 35.275624418905814])

# Buffer by 50 km (adjust if needed: 50000 = 50,000 meters)
roi = point.buffer(50000)

# Map setup
Map = geemap.Map(center=[35.275624418905814, -85.65568438327509], zoom=8)
Map.addLayer(roi, {'color': 'red'}, "ROI Small (50 km buffer)")
# Map.addLayer(tennessee, {}, "Tennessee AOI")
# Map



# # Define your custom ROI
# coords = [
#     [-85.85307177259287, 35.06788356661989],
#     [-85.41087206556162, 35.06788356661989],
#     [-85.41087206556162, 35.34391870157585],
#     [-85.85307177259287, 35.34391870157585],
#     [-85.85307177259287, 35.06788356661989]  # Close the polygon
# ]

# # Create EE Geometry and buffer by 10 km (10000 meters)
# roi = ee.Geometry.Polygon(coords).buffer(100000)


# -----------------------------
# GBIF occurrences for Cerulean Warbler
# -----------------------------
species_name = "Setophaga cerulea"
gbif_data = occurrences.search(scientificName=species_name, country="US", stateProvince="Tennessee", hasCoordinate=True, limit=3000)
records = gbif_data['results']
df = pd.DataFrame(records).dropna(subset=['decimalLatitude', 'decimalLongitude'])

def df_to_fc(row):
    return ee.Feature(ee.Geometry.Point([row['decimalLongitude'], row['decimalLatitude']]), {'occurrence': 1})

presence_fc = ee.FeatureCollection(df.apply(df_to_fc, axis=1).tolist())
Map.addLayer(presence_fc, {'color': 'blue'}, "Presence Points")

# -----------------------------
# Background points
# -----------------------------
background_fc = ee.FeatureCollection.randomPoints(roi, 10000, seed=42).map(lambda f: f.set('occurrence', 0))
training_data = presence_fc.merge(background_fc)
# Map.addLayer(background_fc, {'color': 'red'}, "Background Points")

# df

In [12]:
# -----------------------------
# Landsat 8 NDVI
# -----------------------------
def maskL8sr(image):
    qa = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0)
    )
    return image.updateMask(mask)

LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterDate('2020-01-01', '2020-12-31') \
        .filterBounds(roi) \
        .map(maskL8sr)

def add_ndvi(image):
    return image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

ndvi_image = LS8.map(add_ndvi).median().clip(roi)

proj = ndvi_image.projection()

Map.addLayer(ndvi_image, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, "NDVI")

# -----------------------------
# DEM
# -----------------------------
dem = ee.Image("USGS/SRTMGL1_003").clip(roi).rename("DEM")
Map.addLayer(dem, {'min': 0, 'max': 2000, 'palette': ['white','brown']}, "DEM")

# -----------------------------
# Climate: WorldClim v2 bioclimatic variables (1km resolution)
# -----------------------------
bio1 = ee.Image("WORLDCLIM/V1/BIO").select('bio01').clip(roi).rename("bio01")  # Mean annual temp
bio12 = ee.Image("WORLDCLIM/V1/BIO").select('bio12').clip(roi).rename("bio12")  # Annual precipitation
Map.addLayer(bio1, {}, "bio1")
Map.addLayer(bio12, {}, "bio12")
# -----------------------------
# Forest canopy
# -----------------------------
# # TCC and TCH from Global Forest Canopy Height and Cover
# tcc = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").clip(roi).rename("TCC")  # approximate
# # GEDI products
tcc = ee.ImageCollection("projects/sat-io/open-datasets/facebook/meta-canopy-height").first().clip(roi).rename("TCC");
Map.addLayer(tcc, {}, "tcc")
# Map

# Generate Novel RF to make rh98 GEDI product  wall to wall with assest export

performed one time comment out as already saved as an asset

In [14]:
# # ============================
# # 3. GEDI RH98 data
# # ============================
# def qualityMask(im):
#     return (im.updateMask(im.select('quality_flag').eq(1))
#               .updateMask(im.select('degrade_flag').eq(0)))

# gedi_l2a = (ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY')
#               .map(qualityMask)
#               .select('rh98')
#               .filterDate('2020-01-01', '2020-12-31')
#               .median()
#               .clip(roi))

# # ============================
# # 4. Prepare training data
# # ============================
# predictor_img = ndvi_image

# samples = (predictor_img.addBands(gedi_l2a)
#             .sample(region=roi,
#                     scale=30,
#                     projection=predictor_img.projection(),
#                     geometries=True)
#             .filter(ee.Filter.notNull(['rh98'])))

# # ============================
# # 5. Train RF regression
# # ============================
# rf_rh98 = (ee.Classifier.smileRandomForest(numberOfTrees=50)
#               .setOutputMode('REGRESSION')
#               .train(features=samples,
#                      classProperty='rh98',
#                      inputProperties=predictor_img.bandNames()))

# # ============================
# # 6. Predict RH98
# # ============================
# rh98_prediction = predictor_img.classify(rf_rh98).rename('rh98_pred')

# # ============================
# # 7. Visualization
# # ============================
# ndvi_vis = {'min': 0, 'max': 1, 'palette': ['white','green']}
# gedi_vis = {'min': 1, 'max': 60, 'palette': ['darkred','red','orange','green','darkgreen']}
# pred_vis = {'min': 0, 'max': 40, 'palette': ['white','lightblue','blue','darkblue']}

# Map.centerObject(roi, 10)
# Map.addLayer(ndvi_image, ndvi_vis, 'NDVI')
# Map.addLayer(gedi_l2a, gedi_vis, 'GEDI RH98')
# Map.addLayer(rh98_prediction, pred_vis, 'RF RH98 Prediction (NDVI)')
# Map.addLayer(roi, {}, 'ROI')

# # Map

In [50]:
# asset_folder = "projects/servir-sco-assets/assets/ACI_Colab"
# asset_name = "rh98_prediction"

# # -----------------------------
# # Create and start export task
# # -----------------------------
# task = ee.batch.Export.image.toAsset(
#     image=rh98_prediction,
#     description=f"export_{asset_name}",
#     assetId=f"{asset_folder}/{asset_name}",
#     region=roi,
#     scale=30,
#     maxPixels=1e13
# )
# task.start()

# print(f"✅ Export started → Asset ID: {asset_folder}/{asset_name}")


✅ Export started → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/rh98_prediction


# Generate Novel RF to make PAI GEDI product  wall to wall with assest export

performed one time comment out as already saved as an asset

In [ ]:
# # ============================
# # 1. GEDI L2B PAI data
# # ============================
# def qualityMask(im):
#     return (im.updateMask(im.select('l2b_quality_flag').eq(1))
#               .updateMask(im.select('degrade_flag').eq(0)))

# gedi_l2b = (ee.ImageCollection('LARSE/GEDI/GEDI02_B_002_MONTHLY')
#               .map(qualityMask)
#               .select('pai')
#               .filterDate('2020-01-01', '2020-12-31')
#               .median()
#               .clip(roi))

# # Visualization
# pai_vis = {'min': 1, 'max': 60,
#            'palette': ['darkred','red','orange','green','darkgreen']}
# Map.addLayer(gedi_l2b, pai_vis, 'GEDI L2B PAI')

# print('GEDI L2B PAI image:', gedi_l2b.getInfo())

# # ============================
# # 2. Prepare training data
# # ============================
# predictor_img = ndvi_image  # single predictor: NDVI

# samples_b = (predictor_img.addBands(gedi_l2b)
#               .sample(region=roi,
#                       scale=30,
#                       projection=predictor_img.projection(),
#                       geometries=True)
#               .filter(ee.Filter.notNull(['pai'])))

# # ============================
# # 3. Train RF regression
# # ============================
# rf_pai = (ee.Classifier.smileRandomForest(numberOfTrees=50)
#             .setOutputMode('REGRESSION')
#             .train(features=samples_b,
#                    classProperty='pai',
#                    inputProperties=predictor_img.bandNames()))

# # ============================
# # 4. Predict PAI
# # ============================
# rf_pai_prediction = predictor_img.classify(rf_pai).rename('pai_pred')

# # ============================
# # 5. Visualization
# # ============================
# pai_pred_vis = {'min': 0, 'max': 10,
#                 'palette': ['white','lightblue','blue','darkblue']}

# Map.centerObject(roi, 10)
# Map.addLayer(rf_pai_prediction, pai_pred_vis, 'RF PAI Prediction (NDVI)')

# # Map

In [49]:
# onasset_folder = "projects/servir-sco-assets/assets/ACI_Colab"
# asset_name = "rf_pai_prediction"

# # -----------------------------
# # Create and start export task
# # -----------------------------
# task = ee.batch.Export.image.toAsset(
#     image=rf_pai_prediction,
#     description=f"export_{asset_name}",
#     assetId=f"{asset_folder}/{asset_name}",
#     region=roi,
#     scale=30,
#     maxPixels=1e13
# )
# task.start()

# print(f"✅ Export started → Asset ID: {asset_folder}/{asset_name}")


✅ Export started → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/rf_pai_prediction


# Generate Novel RF to make PADV_z 0-29 GEDI product  wall to wall with assest export

29 unique vertical density map layers generated  

performed one time comment out as already saved as an asset

In [17]:
# # ============================
# # Loop through PAVD_z bands
# # ============================

# # Correct list of bands: pavd_z0 ... pavd_z30
# pavd_bands = [f"pavd_z{i}" for i in range(29)]

# # Dictionaries to hold models and predictions
# rf_models_pavd = {}
# rf_predictions_pavd = {}

# # Inspect first image to check available bands
# first_img = ee.ImageCollection('LARSE/GEDI/GEDI02_B_002_MONTHLY').first()
# available_bands = first_img.bandNames().getInfo()

# for band in pavd_bands:
#     # Skip bands that are not in the collection
#     if band not in available_bands:
#         print(f"⚠️ Band {band} not available, skipping.")
#         continue

#     # Load GEDI L2B band
#     gedi_band = (ee.ImageCollection('LARSE/GEDI/GEDI02_B_002_MONTHLY')
#                    .map(qualityMask)
#                    .select(band)
#                    .filterDate('2020-01-01', '2020-12-31')
#                    .median()
#                    .clip(roi))

#     # Sample NDVI + GEDI band
#     samples = (ndvi_image.addBands(gedi_band)
#                 .sample(region=roi,
#                         scale=30,
#                         projection=ndvi_image.projection(),
#                         geometries=True)
#                 .filter(ee.Filter.notNull([band])))

#     # Train RF regressor
#     rf_model = (ee.Classifier.smileRandomForest(numberOfTrees=50)
#                   .setOutputMode('REGRESSION')
#                   .train(features=samples,
#                          classProperty=band,
#                          inputProperties=ndvi_image.bandNames()))

#     # Predict band from NDVI
#     rf_pred = ndvi_image.classify(rf_model).rename(f"{band}_pred")

#     # Save results
#     rf_models_pavd[band] = rf_model
#     rf_predictions_pavd[band] = rf_pred

# print("✅ Trained and predicted for all available pavd_z bands")


✅ Trained and predicted for all available pavd_z bands


In [18]:
# rf_models_pavd

{'pavd_z0': <ee.classifier.Classifier at 0x7d8dce45d250>,
 'pavd_z1': <ee.classifier.Classifier at 0x7d8dce4b0ce0>,
 'pavd_z2': <ee.classifier.Classifier at 0x7d8dce4b1a90>,
 'pavd_z3': <ee.classifier.Classifier at 0x7d8dce4b29c0>,
 'pavd_z4': <ee.classifier.Classifier at 0x7d8dce4b38f0>,
 'pavd_z5': <ee.classifier.Classifier at 0x7d8dce4bc8c0>,
 'pavd_z6': <ee.classifier.Classifier at 0x7d8dce4bd8b0>,
 'pavd_z7': <ee.classifier.Classifier at 0x7d8dce4be8d0>,
 'pavd_z8': <ee.classifier.Classifier at 0x7d8dce4bf320>,
 'pavd_z9': <ee.classifier.Classifier at 0x7d8dce4c8410>,
 'pavd_z10': <ee.classifier.Classifier at 0x7d8dce4c9490>,
 'pavd_z11': <ee.classifier.Classifier at 0x7d8dce4ca510>,
 'pavd_z12': <ee.classifier.Classifier at 0x7d8dce4cb590>,
 'pavd_z13': <ee.classifier.Classifier at 0x7d8dce4d0650>,
 'pavd_z14': <ee.classifier.Classifier at 0x7d8dce4d16d0>,
 'pavd_z15': <ee.classifier.Classifier at 0x7d8dce4d2210>,
 'pavd_z16': <ee.classifier.Classifier at 0x7d8dce4d3290>,
 'pavd_

In [48]:
# def export_images_to_assets(image_dict, asset_folder, roi, scale=30):
#     """
#     Exports multiple EE images to GEE assets.

#     Parameters:
#     - image_dict: dict of {name: ee.Image} to export
#     - asset_folder: str, folder in your GEE assets (e.g., 'users/your_username')
#     - roi: ee.Geometry, export region
#     - scale: int, resolution in meters (default 30)
#     """
#     for name, img in image_dict.items():
#         asset_id = f"{asset_folder}/{name}"
#         task = ee.batch.Export.image.toAsset(
#             image=img,
#             description=f"export_{name}",
#             assetId=asset_id,
#             region=roi,
#             scale=scale,
#             maxPixels=1e13
#         )
#         task.start()
#         print(f"✅ Started export for {name} → Asset ID: {asset_id}")

# # -----------------------------
# # Example usage
# # -----------------------------
# rf_predictions_pavd = {
#     f"pavd_z{i}": rf_predictions_pavd[f"pavd_z{i}"] for i in range(29)
# }

# asset_folder = "projects/servir-sco-assets/assets/ACI_Colab"  # Replace with your folder
# export_images_to_assets(rf_predictions_pavd, asset_folder, roi)


✅ Started export for pavd_z0 → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/pavd_z0
✅ Started export for pavd_z1 → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/pavd_z1
✅ Started export for pavd_z2 → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/pavd_z2
✅ Started export for pavd_z3 → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/pavd_z3
✅ Started export for pavd_z4 → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/pavd_z4
✅ Started export for pavd_z5 → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/pavd_z5
✅ Started export for pavd_z6 → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/pavd_z6
✅ Started export for pavd_z7 → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/pavd_z7
✅ Started export for pavd_z8 → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/pavd_z8
✅ Started export for pavd_z9 → Asset ID: projects/servir-sco-assets/assets/ACI_Colab/pavd_z9
✅ Started export for pavd_z10 → Asset ID: projects/servir-sco-assets/a

In [19]:
# # Access a specific predicted band
# pred_z5 = rf_predictions_pavd['pavd_z5']  # ee.Image

# print(pred_z5.getInfo())

{'type': 'Image', 'bands': [{'id': 'pavd_z5_pred', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [3, 3], 'origin': [-87, 34], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-85.65568438327509, 35.72556771647989], [-85.81045412939606, 35.707561598626754], [-85.95270944453374, 35.654999793901034], [-86.07099426598994, 35.57212852555563], [-86.1558646071102, 35.465625735472834], [-86.20064834502853, 35.344043671708334], [-86.20194869799064, 35.21710476279884], [-86.15986100165567, 35.09491416795262], [-86.07790304990861, 34.98715567368491], [-85.96268390608564, 34.90233450435306], [-85.82335273729818, 34.84712282956825], [-85.6708783626476, 34.82585307945494], [-85.5172138519597, 34.840192062366924], [-85.37440113624679, 34.88901623606711], [-85.25367027761644, 34.96849574410549], [-85.16458799629929, 35.07238209969832], [-85.11431037629185, 35.192481660616934], [-85.106994371562

In [20]:
# Map.addLayer(pred_z5, {'min': 0, 'max': 10, 'palette': ['white','yellow','green','darkgreen']}, 'pai_z5_pred')
# # Map

In [21]:
# predictors = ee.Image.cat([ndvi_image, dem, bio1, bio12, tcc, rh98_prediction, rf_pai_prediction])#.resample('bilinear').reproject(proj)
# Map.addLayer(predictors, {}, "predictors")
# # Map

# # predictors = predictors.addBands(align(gedi_predictions))

In [ ]:
# -----------------------------
# load the GEE asset
# -----------------------------
pavd_z0= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z0")
pavd_z1= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z1")
pavd_z2= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z2")
pavd_z3= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z3")
pavd_z4= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z4")
pavd_z5= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z5")
pavd_z6= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z6")
pavd_z7= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z7")
pavd_z8= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z8")
pavd_z9= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z9")
pavd_z10= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z10")
pavd_z11= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z11")
pavd_z12= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z12")
pavd_z13= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z13")
pavd_z14= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z14")
pavd_z15= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z15")
pavd_z16= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z16")
pavd_z17= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z17")
pavd_z18= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z18")
pavd_z19= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z19")
pavd_z20= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z20")
pavd_z21= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z21")
pavd_z22= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z22")
pavd_z23= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z23")
pavd_z24= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z24")
pavd_z25= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z25")
pavd_z26= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z26")
pavd_z27= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z27")
pavd_z28= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/pavd_z28")
rf_pai_prediction= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/rf_pai_prediction")
rh98_prediction= ee.Image("projects/servir-sco-assets/assets/ACI_Colab/rh98_prediction")


In [ ]:
# Only use a subset for testing (you can add more later)
predictors = ee.Image.cat([ndvi_image, dem, bio1, bio12, tcc, pavd_z0, pavd_z1, pavd_z2,
                           pavd_z3, pavd_z4, pavd_z5, pavd_z6, pavd_z7, pavd_z8, pavd_z9,
                           pavd_z10, pavd_z11, pavd_z12, pavd_z13, pavd_z14, pavd_z15, pavd_z16,
                           pavd_z17, pavd_z18, pavd_z19, pavd_z20, pavd_z21, pavd_z22, pavd_z23,
                           pavd_z24, pavd_z25, pavd_z26, pavd_z27, pavd_z28, rf_pai_prediction, rh98_prediction])


In [32]:
# # -----------------------------
# # Build lightweight predictors
# # -----------------------------
# def align(img):
#     return img.resample('bilinear')  # <-- no reproject

# # Only use a subset for testing (you can add more later)
# predictors = ee.Image.cat([
#     ndvi_image, dem, bio1, bio12, tcc, rh98_prediction, rf_pai_prediction
# ]).resample('bilinear')

# # Example: add 1–2 PAVD predictions
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z0'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z1'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z2'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z3'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z4'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z5'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z6'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z7'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z8'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z9'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z10'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z11'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z12'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z13'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z14'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z15'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z16'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z17'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z18'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z19'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z20'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z21'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z22'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z23'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z24'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z25'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z26'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z27'])
# predictors = predictors.addBands(rf_predictions_pavd['pavd_z28'])
# # predictors = predictors.addBands(rf_predictions_pavd['pavd_z29'])




✅ Export started → Check Earth Engine Tasks tab or Drive for results.


In [36]:
# ============================================================
# 2. Train MaxEnt model
# ============================================================
# Clip predictors to ROI before sampling
training = predictors.clip(roi).sampleRegions(
    collection=training_data,
    scale=30
)

classifier = ee.Classifier.amnhMaxent() \
    .train(training, 'occurrence', predictors.bandNames())

# Classify probability surface (no tileScale here)
classified = predictors.classify(classifier)

# Clip for visualization/export
probability = classified.select('probability').clip(roi)


In [39]:
# ==============================
# Export image in small tiles
# ==============================
def export_grid(image, roi, grid_size=10, prefix="cerulean_warbler_maxent"):
    # Get bounding box
    coords = roi.bounds().coordinates().getInfo()[0]
    xmin, ymin = coords[0]
    xmax, ymax = coords[2]

    # Compute step sizes
    x_step = (xmax - xmin) / grid_size
    y_step = (ymax - ymin) / grid_size

    tile_id = 0
    for i in range(grid_size):
        for j in range(grid_size):
            x0 = xmin + i * x_step
            x1 = xmin + (i + 1) * x_step
            y0 = ymin + j * y_step
            y1 = ymin + (j + 1) * y_step

            geom = ee.Geometry.Rectangle([x0, y0, x1, y1])
            tile_name = f"{prefix}_tile_{tile_id}"

            task = ee.batch.Export.image.toDrive(
                image=image,
                description=tile_name,
                folder="EarthEngine",
                fileNamePrefix=tile_name,
                region=geom,
                scale=30,
                maxPixels=1e13,
                fileDimensions=512,  # smaller tile size
                shardSize=128
            )
            task.start()
            print(f"🚀 Started export: {tile_name}")
            tile_id += 1


# ==============================
# Run export in 10x10 grid (100 tiles)
# ==============================
export_grid(probability, roi, grid_size=10, prefix="cerulean_warbler_maxent")


🚀 Started export: cerulean_warbler_maxent_tile_0
🚀 Started export: cerulean_warbler_maxent_tile_1
🚀 Started export: cerulean_warbler_maxent_tile_2
🚀 Started export: cerulean_warbler_maxent_tile_3
🚀 Started export: cerulean_warbler_maxent_tile_4
🚀 Started export: cerulean_warbler_maxent_tile_5
🚀 Started export: cerulean_warbler_maxent_tile_6
🚀 Started export: cerulean_warbler_maxent_tile_7
🚀 Started export: cerulean_warbler_maxent_tile_8
🚀 Started export: cerulean_warbler_maxent_tile_9
🚀 Started export: cerulean_warbler_maxent_tile_10
🚀 Started export: cerulean_warbler_maxent_tile_11
🚀 Started export: cerulean_warbler_maxent_tile_12
🚀 Started export: cerulean_warbler_maxent_tile_13
🚀 Started export: cerulean_warbler_maxent_tile_14
🚀 Started export: cerulean_warbler_maxent_tile_15
🚀 Started export: cerulean_warbler_maxent_tile_16
🚀 Started export: cerulean_warbler_maxent_tile_17
🚀 Started export: cerulean_warbler_maxent_tile_18
🚀 Started export: cerulean_warbler_maxent_tile_19
🚀 Started 